In [94]:
parentPath = "/Users/Aran/Documents/workspace/MLExpDir/machine-learning-with-spark"

In [95]:
rawData = sc.textFile("%s/ml-100k/u.data" % parentPath)

In [96]:
rawData.first()

u'196\t242\t3\t881250949'

In [97]:
rawRatings = rawData.map(lambda x: x.split("\t")[:3]) 

In [98]:
rawRatings.first()

[u'196', u'242', u'3']

In [99]:
from pyspark.mllib.recommendation import ALS, Rating

In [100]:
ratings = rawRatings.map(lambda x: Rating(int(x[0]), int(x[1]), float(x[2])))

In [101]:
ratings.first()

Rating(user=196, product=242, rating=3.0)

In [102]:
rank = 50  # The rank of factor matrix
iterations = 10
lamda = 0.01
model = ALS.train(ratings, rank, iterations, lamda)

In [103]:
userFeatures = model.userFeatures()

In [104]:
userFeatures.count()

943

In [105]:
model.predict(789, 123)

3.2791106471124336

In [106]:
ratings

PythonRDD[355] at RDD at PythonRDD.scala:43

In [107]:
from pyspark.rdd import RDD

In [108]:
?RDD.count

In [109]:
userId = 789
K = 10
topKRecs = model.recommendProducts(userId, K)

In [110]:
topKRecs

[Rating(user=789, product=56, rating=6.072091732599815),
 Rating(user=789, product=48, rating=5.6514227162111075),
 Rating(user=789, product=432, rating=5.458713568005688),
 Rating(user=789, product=169, rating=5.404534251556337),
 Rating(user=789, product=611, rating=5.398141708644376),
 Rating(user=789, product=663, rating=5.36904033650295),
 Rating(user=789, product=182, rating=5.2728616441310745),
 Rating(user=789, product=520, rating=5.25214260675391),
 Rating(user=789, product=641, rating=5.245544227971014),
 Rating(user=789, product=346, rating=5.237747159323677)]

In [111]:
movies = sc.textFile("%s/ml-100k/u.item" % parentPath)

In [112]:
titles = movies.map(lambda line: line.split("|")[:2]).map(lambda array: (int(array[0]), array[1])).collectAsMap()

In [113]:
titles[123]

u'Frighteners, The (1996)'

In [114]:
moviesForUser = ratings.keyBy(lambda rating: rating.user).lookup(789)

In [115]:
len(moviesForUser)

33

In [116]:
moviesForUser[0]

Rating(user=789, product=1012, rating=4.0)

In [117]:
topKRateTemp = sorted(moviesForUser, key=lambda rating: rating.rating, reverse=True)[:10]
topKRating = [(titles[rating.product], rating.rating) for rating in topKRateTemp]
for item in topKRating:
    print item 

(u'Godfather, The (1972)', 5.0)
(u'Trainspotting (1996)', 5.0)
(u'Dead Man Walking (1995)', 5.0)
(u'Star Wars (1977)', 5.0)
(u'Swingers (1996)', 5.0)
(u'Leaving Las Vegas (1995)', 5.0)
(u'Bound (1996)', 5.0)
(u'Fargo (1996)', 5.0)
(u'Last Supper, The (1995)', 5.0)
(u'Private Parts (1997)', 4.0)


In [118]:
topKRecsTemp = [(titles[rating.product], rating.rating) for rating in topKRecs]
for item in topKRecsTemp:
    print item

(u'Pulp Fiction (1994)', 6.072091732599815)
(u'Hoop Dreams (1994)', 5.6514227162111075)
(u'Fantasia (1940)', 5.458713568005688)
(u'Wrong Trousers, The (1993)', 5.404534251556337)
(u'Laura (1944)', 5.398141708644376)
(u'Being There (1979)', 5.36904033650295)
(u'GoodFellas (1990)', 5.2728616441310745)
(u'Great Escape, The (1963)', 5.25214260675391)
(u'Paths of Glory (1957)', 5.245544227971014)
(u'Jackie Brown (1997)', 5.237747159323677)


In [119]:
import numpy as np
a = np.array([[1, 2, 3]])
b = np.array([[1], [2], [3]])

In [120]:
b.shape

(3, 1)

In [121]:
b.dot(a)

array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]])

In [122]:
c = np.array([3,4,12])
print np.linalg.norm(c)
d = np.array([1, 2, 3])
print np.dot(c, d)

13.0
47


In [123]:
# Assume the parameters are all 1-d ndarray
def cosineSimilarity(vec1, vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)) 

In [124]:
cosineSimilarity(c, c)

1.0

In [125]:
?RDD.lookup

In [126]:
rdd = sc.parallelize([(1,2), (1,3)],2)
rdd.lookup(1)

[2, 3]

In [127]:
itemId = 567
# The lookup result only has one item, which is the item factor
productFeatures = model.productFeatures()
itemVector = productFeatures.lookup(itemId)[0]

In [128]:
itemVector

array('d', [0.5563952326774597, -1.3195445537567139, 0.5017154812812805, -1.0519071817398071, 0.1207687109708786, 1.0921560525894165, 1.0493335723876953, 0.169920414686203, -0.3367128372192383, 0.29174134135246277, -0.6380590796470642, -0.2363598793745041, -0.11442409455776215, 0.48553165793418884, -0.39993175864219666, -0.2569092810153961, -0.20009008049964905, -0.556082010269165, -0.8290324211120605, 0.4268662631511688, -0.4936608672142029, -0.23795197904109955, -0.01167504582554102, -0.743257462978363, 0.05048343911767006, 0.28117236495018005, 0.43247130513191223, -0.8349613547325134, -0.5094528198242188, -0.14459222555160522, 0.12706108391284943, 0.3807593286037445, 0.6460286378860474, -1.1465355157852173, 0.5278226733207703, -0.4089222550392151, 0.35782989859580994, 0.6055490374565125, -0.07761809229850769, 0.7262181639671326, -0.03011692315340042, 0.7424747943878174, 0.16959147155284882, -0.4313829243183136, -0.7806934714317322, -0.5186501741409302, -0.2848435044288635, 0.3334230

In [129]:
cosineSimilarity(itemVector, itemVector)

1.0000000000000002

In [130]:
sims = productFeatures.map(lambda itemFeature: (itemFeature[0], cosineSimilarity(itemFeature[1], itemVector)))

In [131]:
sims.first()

(8, 0.46869606114429185)

In [132]:
sortedSims = sims.takeOrdered(K, key=lambda x: -x[1])

In [133]:
sortedSims

[(567, 1.0),
 (1376, 0.73533062764883761),
 (109, 0.68299947257653049),
 (184, 0.6743430209778184),
 (563, 0.67019327145837748),
 (895, 0.66592212000650697),
 (288, 0.66528508112292306),
 (201, 0.6651246018441197),
 (295, 0.66485025177381052),
 (1083, 0.65384756783058751)]

In [134]:
titles[itemId]

u"Wes Craven's New Nightmare (1994)"

In [135]:
sortedSims2 = sims.takeOrdered(K+1, key=lambda x: -x[1])
[(titles[item[0]], item[1]) for item in sortedSims2[1:]]

[(u'Meet Wally Sparks (1997)', 0.73533062764883761),
 (u'Mystery Science Theater 3000: The Movie (1996)', 0.68299947257653049),
 (u'Army of Darkness (1993)', 0.6743430209778184),
 (u"Stephen King's The Langoliers (1995)", 0.67019327145837748),
 (u'Scream 2 (1997)', 0.66592212000650697),
 (u'Scream (1996)', 0.66528508112292306),
 (u'Evil Dead II (1987)', 0.66512460184411959),
 (u'Breakdown (1997)', 0.66485025177381052),
 (u'Albino Alligator (1996)', 0.65384756783058751),
 (u'S.F.W. (1994)', 0.65146395677244762)]

In [136]:
actualRating = moviesForUser[0]

In [137]:
predictedRating = model.predict(789, actualRating.product)

In [138]:
predictedRating

3.955014581488941

In [139]:
squaredError = np.square(predictedRating - actualRating.rating)

In [140]:
squaredError

0.0020236878786151474

In [141]:
ratings.first()

Rating(user=196, product=242, rating=3.0)

In [142]:
usersProducts = ratings.map(lambda rating: (rating.user, rating.product))
predictions = model.predictAll(usersProducts).map(lambda rating: ((rating.user, rating.product), rating.rating))

In [143]:
ratingsAndPredictions = ratings.map(lambda rating: ((rating.user, rating.product), rating.rating)).join(predictions)

In [144]:
ratingsAndPredictions.first()

((360, 14), (5.0, 4.949261427802284))

In [145]:
MSE = ratingsAndPredictions.map(lambda x: np.square(x[1][0] - x[1][1])).reduce(lambda x, y: x+y)/ratingsAndPredictions.count()

In [146]:
np.sqrt(MSE)

0.29105688243263866

In [147]:
abc = [1,2,3]
for i, a in enumerate(abc):
    print i,a

0 1
1 2
2 3


In [148]:
def avgPrecisionK(actual, predicted, k):
    predK = predicted[:k]
    score = 0.0
    numHits = 0.0
    for i, p in enumerate(predK):
        if p in actual:
            numHits += 1.0
            score += numHits / (i + 1.0)
    if len(actual) == 0:
        return 1.0
    else:
        return score/min(len(actual), k)

In [149]:
actualMovies = [rating.product for rating in moviesForUser]
predictedMovies = [rating.product for rating in topKRecs]

In [150]:
avgPrecisionK(actualMovies, predictedMovies, 10)

0.0

In [151]:
itemFactors = model.productFeatures().map(lambda idAndFactor: idAndFactor[1]).collect()
itemMatrix = np.array(itemFactors)
itemMatrix.shape

(1682, 50)

In [152]:
imBroadcast = sc.broadcast(itemMatrix)

In [153]:
def getRecommendIds(userIdAndFactor):
    userVector = np.array(userIdAndFactor[1])
    scores = np.dot(imBroadcast.value, userVector)
    sortedWithId = sorted([(score, i) for i, score in enumerate(scores)], key=lambda x: -x[0])
    recommendedIds = [(index+1) for score, index in sortedWithId]
    return (userIdAndFactor[0], recommendedIds)

In [154]:
allRecs = model.userFeatures().map(lambda userIdAndFactor: getRecommendIds(userIdAndFactor))

In [155]:
userMovies = ratings.map(lambda rating: (rating.user, rating.product)).groupBy(lambda x: x[0])

In [156]:
def getMAPK(k):
    return allRecs.join(userMovies).map(lambda x: avgPrecisionK([y[1] for y in x[1][1]], x[1][0], k)).reduce(lambda x, y: x+y)/allRecs.count()

In [157]:
getMAPK(K)

0.03052757158006362

In [158]:
from pyspark.mllib.evaluation import RegressionMetrics

In [159]:
predictedAndTrue = ratingsAndPredictions.map(lambda x: x[1])
regressionMetrics = RegressionMetrics(predictedAndTrue)

In [160]:
print "Mean Squared Error = %f" % regressionMetrics.meanSquaredError
print "Root Mean Squared Error = %f" % regressionMetrics.rootMeanSquaredError

Mean Squared Error = 0.084714
Root Mean Squared Error = 0.291057


In [161]:
from pyspark.mllib.evaluation import RankingMetrics

In [162]:
list(userMovies.first()[1])[:1]

[(2, 292)]

In [163]:
predictedAndTrueForRanking = allRecs.join(userMovies).map(lambda x: (x[1][0], [y[1] for y in x[1][1]]))

In [164]:
rankingMetrics = RankingMetrics(predictedAndTrueForRanking)
print "Mean Average Precision = %f" % rankingMetrics.meanAveragePrecision

Mean Average Precision = 0.072257


In [165]:
getMAPK(2000)

0.07225693265386815